In [3]:
from dtw import *
import numpy as np
from scipy.io.wavfile import read
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import sounddevice as sd
from os.path import dirname, join as pjoin
from scipy.io import wavfile
import scipy.io;
import os
import glob
import webrtcvad
import statistics

import collections
import contextlib
import sys
import wave

vad = webrtcvad.Vad()

import contextlib


capstone_dir = "/Users/ninismacbook/other docs/Y4S1/capstone"

query_sample = []
ref_sample = []
blocksize = 18000

def make_samples(file1, seg1_1, seg1_2, file2, seg2_1, seg2_2):
    
    file1_path = capstone_dir + "/downsampled/" + file1
    file2_path = capstone_dir + "/downsampled/" + file2

    samplerate, query = wavfile.read(file1_path)
    samplerate, ref = wavfile.read(file2_path)

    query_sample = query[int(samplerate*seg1_1): int(samplerate*seg1_2)]
    ref_sample = ref[int(samplerate*seg2_1): int(samplerate*seg2_2)]
    
    return query_sample, ref_sample, samplerate

def play_samples():
    sd.play(query_sample, samplerate = samplerate, blocksize=blocksize, blocking=True)
    sd.play(ref_sample, samplerate = samplerate, blocksize=blocksize, blocking=True);
    
def plot_alignment(alignment):
    alignment.plot(type="threeway"); 
    print('DTW distance: ', alignment.distance);
    print('How much stretching: ', how_much_stretch(alignment))
    
def how_much_stretch(alignment):
    align_len = len(alignment.index1)
    xs = alignment.index1
    ys = alignment.index2
    abs_term = 0
    
    for i in range(align_len):
        if i == 0:
            pass
        if i == align_len-1:
            pass
        else:
            # alignment.M = len of Y axis, alignment.N = len of X axis
            abs_term = abs_term + np.abs((ys[i] - ys[i-1])/(xs[i]-xs[i-1]) - (alignment.N + 1)/(alignment.M+1))
        
        return (1/align_len)*abs_term

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



# Voice activity detection in python (webrtcsav)

In [4]:
def read_wave(path):
    """Reads a .wav file.
    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate
        

# Convert all records into true or false based on whether there is voice activity or not
def speak_pause_rate(audio_path, frame_len):
    
    speech = []
    vad.set_mode(1)

    # 1 pair of \x00\x00 corresponds to 1 frame
    data, samplerate = read_wave(capstone_dir + audio_path)

    speech_segments = [data[i:i+frame_len] for i in range(0, len(data)-frame_len, frame_len)]
    
    print(speech_segments[0])
    for i in speech_segments:
        speech.append(vad.is_speech(i, sample_rate = samplerate))
    print(speech[0])
    
    segment0 = b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
#     print('equal ', segment0 == speech[0])
    
    pause = [not elem for elem in speech]
    return (speech.count(True)/len(speech)), (pause.count(True)/len(speech))

# Does not count the first (before speaker starts speaking) and last pause (after speaker finishes speaking)
def number_of_pauses(audio_path, frame_len):
    speech = []
    
    vad.set_mode(1)
    data, samplerate = read_wave(capstone_dir + '/downsampled/01.wav')
    speech_segments = [data[i:i+frame_len] for i in range(0, len(data)-frame_len, frame_len)]
    
    for i in speech_segments:
        speech.append(vad.is_speech(i, sample_rate = samplerate))
    
#     print(speech)
    for i in range(len(speech)):
        # Find the first True i.e. find the time when speaker starts speaking 
        if(speech[i]==True):
            return('Index of the speech segment when speaker starts speaking', i, speech[i])

In [5]:
print(speak_pause_rate('/downsampled/01.wav', frame_len = 160))        
print(number_of_pauses('/downsampled/01.wav', frame_len = 160))

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
False
(0.8663543675943787, 0.13364563240562138)
('Index of the speech segment when speaker starts speaking', 0, True)


In [6]:
# Run the VAD on 10 ms of silence. The result should be False.
vad.set_mode(1)
sample_rate = 8000
frame_duration = 10  # ms
frame = b'\x00\x00' * int(sample_rate * frame_duration / 1000)
print(frame)
print('Contains speech: ' , vad.is_speech(frame, sample_rate))


b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Contains speech:  True
